In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

In [2]:
print(word_tokenize("This is a test."))


['This', 'is', 'a', 'test', '.']


In [3]:
# Replace 'your_file.csv' with the path to your CSV file
csv_file_path = 'FinSen_US_Categorized_Timestamp.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Display the DataFrame
print(df)

                                                   Title               Tag  \
0                  TSX Slightly Down, Books Weekly Gains      Stock Market   
1                          UnitedHealth Hits 4-week High            stocks   
2                          Cisco Systems Hits 4-week Low            stocks   
3                                 AT&T Hits All-time Low            stocks   
4                             Microsoft Hits 4-week High            stocks   
...                                                  ...               ...   
15529  United States GDP Rises 0.6 percent in the fir...   GDP Growth Rate   
15530  Consumer Price Index 2.6 percent higher than i...    Inflation Rate   
15531  U.S. Federal Reserve Kept Rates Unchanged at 5...     Interest Rate   
15532              Trade Deficit Increases in March 2007  Balance of Trade   
15533  Blackstone boosts IPO after Beijing takes $3bn...              News   

             Time                                            Co

In [4]:
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')
def text_prep(x: str) -> list:
     corp = str(x).lower() 
     corp = re.sub('[^a-zA-Z]+',' ', corp).strip() 
     tokens = word_tokenize(corp)
     words = [t for t in tokens if t not in stop_words]
     lemmatize = [lemma.lemmatize(w) for w in words]
     return lemmatize

In [5]:
preprocess_tag = [text_prep(i) for i in df['Content']]
df["preprocess_txt"] = preprocess_tag

In [6]:
df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

In [7]:
file = open('negative-words.txt', 'r', encoding='latin-1')
neg_words = file.read().split()
file = open('positive-words.txt', 'r', encoding='latin-1')
pos_words = file.read().split()

In [8]:
# nltk.data.path.append('/Users/xiongyuyang/nltk_data')
num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df['pos_count'] = num_pos
num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df['neg_count'] = num_neg

In [9]:
df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 2)
df.head()

,Title,Tag,Time,Content,preprocess_txt,total_len,pos_count,neg_count,sentiment
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,16/07/2023,"TSX Slightly Down, Books Weekly GainsUnited St...","[tsx, slightly, book, weekly, gainsunited, sta...",102,8,6,0.02
1,UnitedHealth Hits 4-week High,stocks,15/07/2023,UnitedHealth Hits 4-week HighUnited States sto...,"[unitedhealth, hit, week, highunited, state, s...",11,0,0,0.00
2,Cisco Systems Hits 4-week Low,stocks,15/07/2023,Cisco Systems Hits 4-week LowUnited States sto...,"[cisco, system, hit, week, lowunited, state, s...",13,0,0,0.00
3,AT&T Hits All-time Low,stocks,15/07/2023,AT&T Hits All-time LowUnited States stocksAT&T...,"[hit, time, lowunited, state, stocksat, decrea...",10,0,0,0.00
4,Microsoft Hits 4-week High,stocks,15/07/2023,Microsoft Hits 4-week HighUnited States stocks...,"[microsoft, hit, week, highunited, state, stoc...",11,0,0,0.00


In [10]:
df['sentiment'] = round(df['pos_count'] / (df['neg_count']+1), 2)
df.head()

,Title,Tag,Time,Content,preprocess_txt,total_len,pos_count,neg_count,sentiment
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,16/07/2023,"TSX Slightly Down, Books Weekly GainsUnited St...","[tsx, slightly, book, weekly, gainsunited, sta...",102,8,6,1.14
1,UnitedHealth Hits 4-week High,stocks,15/07/2023,UnitedHealth Hits 4-week HighUnited States sto...,"[unitedhealth, hit, week, highunited, state, s...",11,0,0,0.00
2,Cisco Systems Hits 4-week Low,stocks,15/07/2023,Cisco Systems Hits 4-week LowUnited States sto...,"[cisco, system, hit, week, lowunited, state, s...",13,0,0,0.00
3,AT&T Hits All-time Low,stocks,15/07/2023,AT&T Hits All-time LowUnited States stocksAT&T...,"[hit, time, lowunited, state, stocksat, decrea...",10,0,0,0.00
4,Microsoft Hits 4-week High,stocks,15/07/2023,Microsoft Hits 4-week HighUnited States stocks...,"[microsoft, hit, week, highunited, state, stoc...",11,0,0,0.00


In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sent = SentimentIntensityAnalyzer()
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in df['Content']]
df['sentiment_score'] = polarity
df.head()

,Title,Tag,Time,Content,preprocess_txt,total_len,pos_count,neg_count,sentiment,sentiment_score
0,"TSX Slightly Down, Books Weekly Gains",Stock Market,16/07/2023,"TSX Slightly Down, Books Weekly GainsUnited St...","[tsx, slightly, book, weekly, gainsunited, sta...",102,8,6,1.14,0.81
1,UnitedHealth Hits 4-week High,stocks,15/07/2023,UnitedHealth Hits 4-week HighUnited States sto...,"[unitedhealth, hit, week, highunited, state, s...",11,0,0,0.00,0.27
2,Cisco Systems Hits 4-week Low,stocks,15/07/2023,Cisco Systems Hits 4-week LowUnited States sto...,"[cisco, system, hit, week, lowunited, state, s...",13,0,0,0.00,-0.27
3,AT&T Hits All-time Low,stocks,15/07/2023,AT&T Hits All-time LowUnited States stocksAT&T...,"[hit, time, lowunited, state, stocksat, decrea...",10,0,0,0.00,-0.27
4,Microsoft Hits 4-week High,stocks,15/07/2023,Microsoft Hits 4-week HighUnited States stocks...,"[microsoft, hit, week, highunited, state, stoc...",11,0,0,0.00,0.27
